<a href="https://colab.research.google.com/github/rahilvir/Computer-Vision-Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# zip files were just much faster to uplaod to google drive
!unzip /content/drive/MyDrive/data.zip

In [ ]:
# this code made sure each class had the same number of images
import os
import random
from pathlib import Path
from collections import defaultdict

def balance_classes(data_dir):
    for split in ["train", "val"]:
        split_dir = Path(data_dir) / split
        class_counts = {}

        #count images in each class folder
        for class_dir in split_dir.iterdir():
            if class_dir.is_dir():
                class_counts[class_dir] = len(list(class_dir.glob("*.*")))

        # find minimum count
        min_count = min(class_counts.values())
        print(f"Minimum image count in {split}: {min_count}")

        # balance each class
        for class_dir, count in class_counts.items():
            if count > min_count:
                images = list(class_dir.glob("*.*"))
                random.shuffle(images)  # Shuffle to ensure random selection
                for img in images[min_count:]:  # Keep only min_count images
                    img.unlink()  # Delete the extra image file
                print(f"Reduced {class_dir} to {min_count} images")

data_path = '/content/data'
balance_classes(data_path)

In [ ]:
from ultralytics import YOLO

model = YOLO('content/yolo11x-cls.pt')

model.train(data='data', epochs=10, imgsz=256) #image size is to make sure our individual images were of the same size as the rest of the dataset

In [ ]:
!zip -r runs.zip /content/runs #just to download our 'runs' folder from runtime
from google.colab import files
files.download('runs.zip')

In [ ]:
model= YOLO('runs/classify/train/weights/best.pt')
metrics = model.val()
print(metrics)

In [ ]:
# testing the model
results=model('/content/data/test/Apple Scab/0261a6e4-21f8-481a-8827-b674e6955644___FREC_Scab 3055_270deg.JPG', save=True)
for r in results:
    print(r.probs)